In [ ]:
import re
import requests
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import xlrd
import json

import pandas as pd
links = pd.read_excel("Any Excel file", usecols= range(0,3)) 
company = links["Corresponding column name of link "]
links = links["column name with urls"]
FILE_NAME = 'output.json' # Name of output file

with open(FILE_NAME,"w") as file:
  json.dump([],file)


duplicate = deque()
com = deque()
for i in range(len(links)):
  link = links[i]
  duplicate.append(link)
#duplicate


def save(x):
    with open(FILE_NAME,'w') as f:
        json.dump(x,f)

scraped = set()
emails = set()
phoneno = set()
final_emails = []
final_set = []
k = 0
for t in range(len(duplicate)):
  original_url = duplicate[t]
  unscraped = deque([original_url])
  emails.clear()
  count = 0
  while len(unscraped) and count < 10:
       

        url = unscraped.popleft()  
        scraped.add(url)
        try:
          parts = urlsplit(url)
        except:
          continue  
        base_url = "{0.scheme}://{0.netloc}".format(parts)
        if '/' in parts.path:
          path = url[:url.rfind('/')+1]
        else:
          path = url

        print("Crawling URL %s" % url)
        try:
            response = requests.get(url)
        except:
            continue

        soup = BeautifulSoup(response.text, 'lxml')
        
        for anchor in soup.find_all("a"):
          if anchor.text != "Contact Us" and anchor.text != "Contact"and anchor.text != "About Us" and anchor.text != "About" :
            continue
          
          if "href" in anchor.attrs:
            link = anchor.attrs["href"]
          else:
            link = ''
          
          
          if link.startswith('/'):
                link = base_url + link
            
            
          elif not link.startswith('http'):
                
                link = path + link

          len1 = len(link)
          if(link[len1-1] == '#' ):
            break     


          if not link.endswith(".gz"):
            if not link in unscraped and not link in scraped:
                unscraped.append(link)
          
        # For Phone Number
        new_phonea = re.findall(r'\(?\b[2-9][0-9]{2}\)?[-][2-9][0-9]{2}[-][0-9]{4}\b', response.text)
        phoneno.update(new_phonea)
        
        # For Emails
        new_emails_com = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.com", response.text, re.I))
        new_emails_org = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.org", response.text, re.I))
        new_emails_au = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.au", response.text, re.I))
        new_emails_in = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.in", response.text, re.I))
        new_emails_af = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.af", response.text, re.I))
        new_emails_ar = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.ar", response.text, re.I))
        new_emails_us = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.us", response.text, re.I))
        emails.update(new_emails_org)
        emails.update(new_emails_com)
        emails.update(new_emails_au)
        emails.update(new_emails_af)
        emails.update(new_emails_ar)
        emails.update(new_emails_in)
        emails.update(new_emails_us)
        count += 1
  # List to store multiple emails
  e = []
  for x in emails:
    print(x)
    e.append(x)
  # List to store multiple phone numbers
  b = []
  for x in phoneno:
    print(x)
    b.append(x)
 
 ## Storing the details in JSON file defined above
  final_emails.append(e)
  with open(FILE_NAME) as file:
    data = json.load(file)
    organisation = {"Name" : company[k],"email" : e,"phoneNo" : b} # Code to store the json objects for easier conversion to csv file later
    print(organisation)
    data.append(organisation)
    save(data)
  k += 1   
print(final_emails)
    




In [ ]:
### Print the json file


with open('output.json', 'r') as raw_output:
    data = raw_output.read()
    output = json.loads(data)
print(output)